In [1]:
import sys, os
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
from survey import Survey, nine_to_county, purp_num_to_name18, purp_num_to_name23, mode_num_to_name23, county_order
from processed_survey import ProcessedSurvey
from survey_completeness import flag_complete, PERSON_REQS, HH_REQS, DAY_REQS, TRIP_REQS
sys.path.insert(0, r'Y:\champ\util\pythonlib-migration\master_versions\misc_utils')
from df_utils import df_to_excel

In [2]:
SURVEY_2023 = {'household':{'filepath_or_buffer':r'..\..\..\..\Review_20241127\flagged\hh.csv'},
               'person':{'filepath_or_buffer':r'..\..\..\..\Review_20241127\flagged\person.csv'},
               'day':{'filepath_or_buffer':r'..\..\..\..\Review_20241127\flagged\day.csv'},
               'trip':{'filepath_or_buffer':r'..\..\..\..\Review_20241127\flagged\trip.csv'},
               'vehicle':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20241127\vehicle.csv'},
               'location':{'filepath_or_buffer':r'..\..\..\..\Deliverable_20241127\location.csv'},
               }
OUTDIR = r'..\..\..\..\Review_20241127'
COUNTIES = r'Q:\GIS\Boundaries\Counties\Counties.shp'
FIPS_COUNTIES   =['001','013','041','055','075','081','085','095','097']

In [3]:
survey_args = {'dir':r'',
               'hh':    {'file':'survey2023_hrecx_rewt_base2023.dat',
                         'sep':' ',
                         'weight':'hhexpfac'},
               'person':{'file':'survey2023_precx_rewt_base2023.dat',
                         'sep':' ',
                         'weight':'psexpfac'},
               'day':   {'file':'survey2023_pdayx_rewt_base2023.dat',
                         'sep':' ',
                         'weight':'pdexpfac'},
               'trip':  {'file':'survey2023_tripx_rewt_base2023.dat',
                         'sep':' ',
                         'weight':'trexpfac'},
               'tour':  {'file':'survey2023_tourx_rewt_base2023.dat',
                         'sep':' ',
                         'weight':'toexpfac'},
#               'raw_dir': r'',
#               'raw_hh':     {'file':'hh.csv',
#                              'weight':'hh_weight'},
#               'raw_person': {'file':'person.csv',
#                              'weight':'person_weight'},
#               'raw_day':    {'file':'day.csv',
#                              'weight':'day_weight'},
#               'raw_trip':   {'file':'trip.csv',
#                              'weight':'trip_weight'},
             }

In [4]:
s23 = Survey(**SURVEY_2023)

In [5]:
ps23 = ProcessedSurvey(**survey_args)

In [6]:
flagged = flag_complete(s23, 
                    hh_unit='family',
                    concurrent_days=0,
                    concurrent_weighted_days=0)

Q:\Data\Surveys\HouseholdSurveys\MTC-SFCTA2022\travel_diary_survey_scripts\2022_v00c\99-review\notebooks\survey_completeness.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person.drop(columns=['sfcta_num_days_complete'], inplace=True)


In [7]:
ps23 = ProcessedSurvey(**survey_args)
hh2 = ps23.hh.data.rename(columns={'hhno':'hh_id'})
hh2['tour_complete'] = 1
person2 = ps23.person.data.rename(columns={'hhno':'hh_id','pno':'person_num'})
person2['tour_complete'] = 1
day2 = ps23.day.data.rename(columns={'hhno':'hh_id','pno':'person_num','day':'travel_dow'})
day2['tour_complete'] = 1
trip2 = ps23.trip.data.rename(columns={'hhno':'hh_id','pno':'person_num','day':'travel_dow','tsvid':'trip_num'})
trip2['tour_complete'] = 1
tour2 = ps23.tour.data
tour2['tour_complete'] = 1

In [8]:
hhcols = ['sfcta_hh_survey_complete','sfcta_hh_persons_complete',
          'sfcta_hh_concurrent_complete','sfcta_hh_concurrent_weighted_complete',
          'sfcta_hh_complete']
hh = pd.merge(flagged.hh, hh2[['hh_id','tour_complete']], how='left').fillna(0)
hh['all_complete'] = hh['sfcta_hh_complete'] * hh['tour_complete']
person = pd.merge(flagged.person, person2[['hh_id','person_num','tour_complete']], how='left').fillna(0)
person = pd.merge(flagged.hh[['hh_id']+hhcols], person)
person['all_complete'] = person['sfcta_hh_complete'] * person['tour_complete']
day = pd.merge(flagged.day, day2[['hh_id','person_num','travel_dow','tour_complete']], how='left').fillna(0)
day = pd.merge(flagged.hh[['hh_id']+hhcols], day)
day['all_complete'] = day['sfcta_hh_complete'] * day['tour_complete']
trip = pd.merge(flagged.trip, trip2[['hh_id','person_num','travel_dow','trip_num','tour_complete']], how='left').fillna(0)
trip = pd.merge(flagged.hh[['hh_id','diary_platform']+hhcols], trip)
trip['all_complete'] = trip['sfcta_hh_complete'] * trip['tour_complete']

In [9]:
day = pd.merge(day, person[['person_id','diary_platform','age']])

In [10]:
day['no_travel'] = day['num_trips'].eq(0) * 1

In [11]:
day['is_typwkday'] = day['travel_dow'].isin([2,3,4]) * 1
day['is_adult'] = day['age'].ge(4) * 1

In [12]:
ntdays = day.loc[day['is_complete'].eq(1)].pivot_table(index=['is_typwkday','diary_platform','age','is_adult'], columns='no_travel', values='day_weight', aggfunc='count')
ntdays #.divide(ntdays.sum(axis=1), axis='rows')

no_travel                                     0       1
is_typwkday diary_platform age is_adult                
0           rmove          4   1         1518.0   321.0
                           5   1         8384.0  1310.0
                           6   1         7832.0  1224.0
                           7   1         5231.0   872.0
                           8   1         5063.0  1151.0
                           9   1         4284.0  1151.0
                           10  1         1219.0   442.0
                           11  1           80.0    95.0
1           browser        1   0           55.0    20.0
                           2   0          147.0    23.0
                           3   0           34.0     5.0
                           4   1           86.0    38.0
                           5   1          228.0    75.0
                           6   1          272.0    74.0
                           7   1          256.0    67.0
                           8   1          322.0    90.0
                           9   1          310.0   115.0
                           10  1          154.0    95.0
                           11  1           37.0    70.0
            call           1   0            NaN     5.0
                           2   0            2.0     3.0
                           3   0            2.0     1.0
                           4   1            4.0     1.0
                           5   1            4.0     7.0
                           6   1            9.0     1.0
                           7   1           17.0     5.0
                           8   1           49.0     8.0
                           9   1           75.0    21.0
                           10  1           53.0    26.0
                           11  1           20.0     7.0
            rmove          1   0          339.0   154.0
                           2   0          801.0   277.0
                           3   0          120.0    54.0
                           4   1         1234.0   169.0
                           5   1         6317.0  1031.0
                           6   1         6054.0   842.0
                           7   1         4081.0   556.0
                           8   1         3962.0   766.0
                           9   1         3354.0   772.0
                           10  1          992.0   283.0
                           11  1           88.0    49.0

In [13]:
hhdf = (hh.groupby('diary_platform')[HH_REQS+hhcols+['tour_complete','all_complete']]
        .sum().T)
hhdf.loc['total',:] = hh.groupby('diary_platform').size().T

In [14]:
psdf = (person.groupby('diary_platform')[PERSON_REQS+hhcols+['tour_complete','all_complete']]
        .sum().T)
psdf.loc['total',:] = person.groupby('diary_platform').size().T

In [15]:
pddf = (day.groupby('diary_platform')[DAY_REQS+hhcols+['tour_complete','all_complete']]
        .sum().T)
pddf.loc['total',:] = day.groupby('diary_platform').size().T

In [16]:
trip

,hh_id,diary_platform,sfcta_hh_survey_complete,sfcta_hh_persons_complete,sfcta_hh_concurrent_complete,sfcta_hh_concurrent_weighted_complete,sfcta_hh_complete,trip_id,first_travel_date,last_travel_date,...,arrive_second_complete,o_lat_complete,o_lon_complete,d_lat_complete,d_lon_complete,sfcta_is_complete,has_weight,has_weight_rmove,tour_complete,all_complete
0,23000075,browser,1,1,1,1,1,2300007501001,2023-05-17,2023-05-17,...,1,1,1,1,1,1,1,0,1.0,1.0
1,23000075,browser,1,1,1,1,1,2300007501002,2023-05-17,2023-05-17,...,1,1,1,1,1,1,1,0,1.0,1.0
2,23000075,browser,1,1,1,1,1,2300007501003,2023-05-17,2023-05-17,...,1,1,1,1,1,1,1,0,1.0,1.0
3,23000075,browser,1,1,1,1,1,2300007501004,2023-05-17,2023-05-17,...,1,1,1,1,1,1,1,0,1.0,1.0
4,23000075,browser,1,1,1,1,1,2300007501005,2023-05-17,2023-05-17,...,1,1,1,1,1,1,1,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365826,23803687,rmove,1,1,1,1,1,2380368701038,2023-12-01,2023-12-07,...,1,1,1,1,1,1,1,1,0.0,0.0
365827,23803687,rmove,1,1,1,1,1,2380368701039,2023-12-01,2023-12-07,...,1,1,1,1,1,1,1,1,0.0,0.0
365828,23803687,rmove,1,1,1,1,1,2380368701040,2023-12-01,2023-12-07,...,1,1,1,1,1,1,1,1,0.0,0.0
365829,23803687,rmove,1,1,1,1,1,2380368701041,2023-12-01,2023-12-07,...,1,1,1,1,1,1,1,1,0.0,0.0


In [17]:
trdf = (trip.groupby('diary_platform')[TRIP_REQS+hhcols+['tour_complete','all_complete']]
        .sum().T)
trdf.loc['total',:] = trip.groupby('diary_platform').size().T

In [18]:
hhdf

diary_platform,browser,call,rmove
num_workers_complete,1096.0,243.0,6919.0
num_adults_complete,1096.0,243.0,6919.0
num_kids_complete,1096.0,243.0,6919.0
num_workers_complete,1096.0,243.0,6919.0
num_vehicles_complete,1096.0,243.0,6919.0
home_lat_complete,1096.0,243.0,6919.0
home_lon_complete,1096.0,243.0,6919.0
sfcta_hh_survey_complete,1096.0,243.0,6919.0
sfcta_hh_persons_complete,1043.0,237.0,6716.0
sfcta_hh_concurrent_complete,1057.0,238.0,6754.0


In [19]:
psdf

diary_platform,browser,call,rmove
race_complete,2573.0,320.0,12317.0
ethnicity_complete,2573.0,320.0,12317.0
gender_complete,2573.0,320.0,12317.0
age_complete,2573.0,320.0,12317.0
employment_complete,2573.0,320.0,12317.0
telework_complete,2573.0,320.0,12317.0
school_loc_complete,2573.0,320.0,12317.0
work_park_complete,2554.0,319.0,12271.0
sfcta_hh_survey_complete,2573.0,320.0,12317.0
sfcta_hh_persons_complete,2373.0,296.0,11757.0


In [20]:
pddf

diary_platform,browser,call,rmove
telecommute_time_complete,2541.0,315.0,84629.0
sfcta_hh_survey_complete,2573.0,320.0,86219.0
sfcta_hh_persons_complete,2373.0,296.0,82299.0
sfcta_hh_concurrent_complete,2429.0,297.0,82845.0
sfcta_hh_concurrent_weighted_complete,2573.0,320.0,86219.0
sfcta_hh_complete,2373.0,296.0,82124.0
tour_complete,2573.0,320.0,72472.0
all_complete,2373.0,296.0,69640.0
total,2573.0,320.0,86219.0


In [21]:
person[['diary_platform','sfcta_hh_complete','sfcta_person_survey_complete','sfcta_person_complete','tour_complete']].groupby('diary_platform').sum()

,sfcta_hh_complete,sfcta_person_survey_complete,sfcta_person_complete,tour_complete
diary_platform,,,,
browser,2373,2554,2513,2573
call,296,319,312,320
rmove,11732,12271,12097,12317


In [22]:
day[['diary_platform','sfcta_hh_complete','sfcta_day_survey_complete','sfcta_day_trips_complete','tour_complete']].groupby('diary_platform').sum()

,sfcta_hh_complete,sfcta_day_survey_complete,sfcta_day_trips_complete,tour_complete
diary_platform,,,,
browser,2373,2541,2564.0,2573.0
call,296,315,318.0,320.0
rmove,82124,84629,79763.0,72472.0


In [23]:
pday = pd.merge(person[['hh_id','person_num','diary_platform','sfcta_hh_complete','sfcta_person_survey_complete','sfcta_person_complete']],
                day[['hh_id','person_num','travel_dow','sfcta_day_survey_complete','sfcta_day_trips_complete','tour_complete']])
pday['all_complete'] = 1
for f in ['sfcta_hh_complete','sfcta_person_survey_complete','sfcta_person_complete','sfcta_day_survey_complete','sfcta_day_trips_complete','tour_complete']:
    pday['all_complete'] = pday['all_complete'] * pday[f]

In [24]:
pday.loc[pday['travel_dow'].isin([2,3,4])].groupby(['diary_platform'])[['sfcta_hh_complete','sfcta_person_survey_complete','sfcta_person_complete','sfcta_day_survey_complete','sfcta_day_trips_complete','tour_complete','all_complete']].sum().T

diary_platform,browser,call,rmove
sfcta_hh_complete,2373.0,296.0,35196.0
sfcta_person_survey_complete,2554.0,319.0,36813.0
sfcta_person_complete,2513.0,312.0,36291.0
sfcta_day_survey_complete,2541.0,315.0,36365.0
sfcta_day_trips_complete,2564.0,318.0,34526.0
tour_complete,2573.0,320.0,32295.0
all_complete,2373.0,296.0,30766.0


In [25]:
pday.loc[pday['travel_dow'].isin([2,3,4])].groupby(['diary_platform']).size()

diary_platform
browser     2573
call         320
rmove      36951
dtype: int64

In [26]:
pddf.loc['sfcta_hh_complete'] / pddf.loc['total']

diary_platform
browser    0.922270
call       0.925000
rmove      0.952505
dtype: float64